In [5]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

station = 'BHP'
year = '2022'
channels = ['BHN','BHE','BHZ']

for channel in channels:
    file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-01-09.mseed'
    stream = read(file_path)
    print(stream)

1 Trace(s) in Stream:
CI.BHP..BHN | 2022-01-09T00:00:00.019538Z - 2022-01-09T23:59:59.994538Z | 40.0 Hz, 3456000 samples
1 Trace(s) in Stream:
CI.BHP..BHE | 2022-01-09T00:00:00.019536Z - 2022-01-09T23:59:59.994536Z | 40.0 Hz, 3456000 samples
1 Trace(s) in Stream:
CI.BHP..BHZ | 2022-01-09T00:00:00.019538Z - 2022-01-09T23:59:59.994538Z | 40.0 Hz, 3456000 samples


In [7]:
stream[0].stats

         network: CI
         station: BHP
        location: 
         channel: BHZ
       starttime: 2022-01-09T00:00:00.019538Z
         endtime: 2022-01-09T23:59:59.994538Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 3456000
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'D', 'number_of_records': 16485, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 8440320})

In [8]:
import os
import logging
from obspy import read, Stream, UTCDateTime
import numpy as np
from obspy import Trace

def format_duration(duration):
    """Convert duration from seconds to hours:minutes format."""
    hours = int(duration // 3600)
    minutes = int((duration % 3600) // 60)
    return f"{hours:02d}:{minutes:02d}"

def save_processed_trace(trace, original_path):
    """Save the processed trace to a new file with '_processed' appended to the original name."""
    directory, filename = os.path.split(original_path)
    name, ext = os.path.splitext(filename)
    new_filename = f"{name}_processed{ext}"
    new_path = os.path.join(directory, new_filename)
    trace.write(new_path, format='MSEED')
    logging.info(f"Saved processed trace to: {new_path}")

def process_waveform(file_path):
    try:
        stream = read(file_path)
        #print(f"Number of traces: {len(stream)}")
        if len(stream) > 1:
            print(stream)
        
        
        stream.merge(method=1, fill_value=0)
        duration = stream[0].stats.endtime - stream[0].stats.starttime
        duration_str = format_duration(duration)
        logging.info(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")
        #print(f"Stream goes from {stream[0].stats.starttime} to {stream[0].stats.endtime} (duration: {duration_str})")

        # Save the processed trace
        save_processed_trace(stream[0], file_path)

    except Exception as e:
        logging.error(f"Error processing file {stream[0]}: {str(e)}")
        print(f"Error processing file {stream[0]}: {str(e)}")



base_dir = f"/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}"

logging.basicConfig(filename=f"{year}_{station}_trace_preprocessing.log", level=logging.INFO, format='%(asctime)s - %(message)s', filemode='w')
# Base directory to start searching


print(f'processing year {year}')
# Collect all mseed files recursively in a list
file_list = []

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".mseed"):
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Sort the list of files alphabetically
file_list.sort()
print(f"In total {len(file_list)} files")

# Process each file that has not been processed yet, total 1095 in 2021
for file_path in file_list:
    # Skip already processed files
    if "_processed" in os.path.basename(file_path):
        logging.info(f"Skipping already processed file: {os.path.basename(file_path)}")
        continue
    
    #print(f"Processing file {file_path}")
    logging.info(f"Processing file {os.path.basename(file_path)}")
    process_waveform(file_path)

processing year 2022
In total 1095 files
2 Trace(s) in Stream:
CI.BHP..BHE | 2022-01-13T00:00:00.019538Z - 2022-01-13T23:43:21.044538Z | 40.0 Hz, 3416042 samples
CI.BHP..BHE | 2022-01-13T23:45:09.344538Z - 2022-01-13T23:59:59.994538Z | 40.0 Hz, 35627 samples
2 Trace(s) in Stream:
CI.BHP..BHE | 2022-01-14T00:00:00.019536Z - 2022-01-14T00:07:04.719536Z | 40.0 Hz, 16989 samples
CI.BHP..BHE | 2022-01-14T00:09:19.019536Z - 2022-01-14T23:59:59.994536Z | 40.0 Hz, 3433640 samples
39 Trace(s) in Stream:

CI.BHP..BHE | 2022-02-23T00:00:00.019538Z - 2022-02-23T03:49:29.219538Z | 40.0 Hz, 550769 samples
...
(37 other traces)
...
CI.BHP..BHE | 2022-02-23T17:29:22.144538Z - 2022-02-23T23:59:59.994538Z | 40.0 Hz, 937515 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
2 Trace(s) in Stream:
CI.BHP..BHE | 2022-02-25T00:00:00.019538Z - 2022-02-25T18:19:00.569538Z | 40.0 Hz, 2637623 samples
CI.BHP..BHE | 2022-02-25T18:20:22.994538Z - 2022-02-25T23:59:59.994538Z | 40.0 Hz, 815081 

In [9]:
channel = 'BHN' #'BHE','BHZ'


file_path = f'/home/gridsan/mknuth/01_Seismic_Wave_Data_Prediction/01_Data/01_Seismic_Wave_Data/{year}/{station}/{channel}/{station}_{channel}_{year}-07-23.mseed'
stream = read(file_path)
print(stream)

1 Trace(s) in Stream:
CI.BHP..BHN | 2022-07-23T00:00:00.019536Z - 2022-07-23T23:59:59.994536Z | 40.0 Hz, 3456000 samples
